In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.0 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [2]:
!pip install emoji==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49720 sha256=d02aad3564d9fe99464fcdff1bbe37194c84ef62b42b46969c333264e17e65ce
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji
ERROR: Operation cancelled by user


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import LongformerModel, LongformerTokenizer
from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from transformers import DistilBertModel, DistilBertTokenizerFast
import tensorflow as tf
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from transformers import EarlyStoppingCallback
from torch.optim import AdamW
from transformers import AutoModel

Exception ignored in: <function _xla_gc_callback at 0x7f4c3d76c160>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Thesis/commentbig5sample.csv", encoding='latin-1')

In [ ]:
# Drop colums
df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)

In [ ]:

# # Initialize tokenizer

# tokenizer
tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base', normalization=True)  

# Preprocess tweets: tokenize and pad/truncate
max_length = 64

df['body'] = df['body'].apply(lambda x: tokenizer.encode_plus(x, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt'))



# Split into train and test first
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

# Split df_train into train and validation
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)

In [9]:
# Define a PyTorch dataset
class PersonalityDataset(Dataset):
    def __init__(self, tweets, targets):
        self.tweets = tweets
        self.targets = targets

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, idx):
      input_ids = torch.tensor(self.tweets[idx]['input_ids'], dtype=torch.long).squeeze(0)
      attention_mask = torch.tensor(self.tweets[idx]['attention_mask'], dtype=torch.long).squeeze(0)
      targets = torch.tensor(self.targets[idx], dtype=torch.float)

      return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'targets': targets
      }

In [10]:
class BERTweetForPersonalityTraits(torch.nn.Module):
    def __init__(self):
        super(BERTweetForPersonalityTraits, self).__init__()
        self.bertweet = AutoModel.from_pretrained('vinai/bertweet-base')
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 5)  # Update the input size to match the BERTweet hidden size

    def forward(self, input_ids, attention_mask):
        outputs = self.bertweet(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # Use the [CLS] token representation
        output = self.dropout(pooled_output)
        output = self.linear(output)
        return output


In [11]:
# Prepare data loaders
batch_size = 16
train_dataset = PersonalityDataset(df_train['body'].tolist(), df_train[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']].values)
val_dataset = PersonalityDataset(df_val['body'].tolist(), df_val[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']].values)
test_dataset = PersonalityDataset(df_test['body'].tolist(), df_test[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Instantiate the model and define optimizer and loss function

model = BERTweetForPersonalityTraits()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.MSELoss()

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
# Training loop
epochs = 10

# Early stopping parameters
patience = 3
best_val_loss = float('inf')
patience_counter = 0
accumulation_steps = 16

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad() if i % accumulation_steps == 0 else None

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)

# Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, targets)

        # Backward pass
        loss = loss / accumulation_steps
        loss.backward()
        if (i+1) % accumulation_steps == 0 or i+1 == len(train_loader):  # update on last step even if accumulation steps have not been reached
            optimizer.step()
            optimizer.zero_grad()  # make sure to reset gradient after updating

        total_loss += loss.item()

        # Delete unnecessary tensors to save GPU memory
        del input_ids
        del attention_mask
        del targets
        del outputs
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Train loss at epoch {epoch + 1}: {avg_train_loss}")

    # Validation loop
    model.eval()
    val_total_loss = 0
    all_predictions = [[] for _ in range(5)]  # Separate predictions for each trait
    all_targets = [[] for _ in range(5)]  # Separate targets for each trait

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs, targets)

            val_total_loss += loss.item()

            # Split predictions and targets for each trait
            for i in range(5):
                all_predictions[i].extend(outputs[:, i].cpu().numpy())
                all_targets[i].extend(targets[:, i].cpu().numpy())

    avg_val_loss = val_total_loss / len(val_loader)
    print(f"Validation loss at epoch {epoch + 1}: {avg_val_loss}")

    # Check if early stopping conditions are met
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0  # reset counter
        # save the best model
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1  # increment counter
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break  # break out of the training loop


# After the validation loop...
trait_names = ["Extroversion", "Neuroticism", "Agreeableness", "Conscientiousness", "Openness"]

for i in range(5):
    predictions = all_predictions[i]
    targets = all_targets[i]

    mse = mean_squared_error(targets, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(targets, predictions)

    print(f"For {trait_names[i]}:")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"R^2: {r2}\n")


<ipython-input-9-217b49c1381b>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(self.tweets[idx]['input_ids'], dtype=torch.long).squeeze(0)
<ipython-input-9-217b49c1381b>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(self.tweets[idx]['attention_mask'], dtype=torch.long).squeeze(0)


Train loss at epoch 1: 0.023988686298782175
Validation loss at epoch 1: 0.24776346130030497
Train loss at epoch 2: 0.014447106776589697
Validation loss at epoch 2: 0.15021509623953275
Train loss at epoch 3: 0.010060020307586952
Validation loss at epoch 3: 0.12190112045833043
Train loss at epoch 4: 0.008192636936225675
Validation loss at epoch 4: 0.12083156034350395
Train loss at epoch 5: 0.007873522109267386
Validation loss at epoch 5: 0.11821890888469559
Train loss at epoch 6: 0.007551881450821053
Validation loss at epoch 6: 0.11127020312207085
Train loss at epoch 7: 0.007140525100244717
Validation loss at epoch 7: 0.10413748451641627
Train loss at epoch 8: 0.0069573827680539
Validation loss at epoch 8: 0.10061249669109072
Train loss at epoch 9: 0.006879125831818039
Validation loss at epoch 9: 0.10003813675471715
Train loss at epoch 10: 0.006809473867443475
Validation loss at epoch 10: 0.10006134584546089
For Extroversion:
MSE: 0.1090237945318222
RMSE: 0.33018749952316284
R^2: -0.0390

In [13]:
# Evaluation on test set
model.eval()
test_total_loss = 0
all_predictions = [[] for _ in range(5)]  # Separate predictions for each trait
all_targets = [[] for _ in range(5)]  # Separate targets for each trait

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, targets)

        test_total_loss += loss.item()

        # Split predictions and targets for each trait
        for i in range(5):
            all_predictions[i].extend(outputs[:, i].cpu().numpy())
            all_targets[i].extend(targets[:, i].cpu().numpy())

trait_names = ["Ext", "Neu", "Agre", "Con", "Ope"]

for i in range(5):
    predictions = all_predictions[i]
    targets = all_targets[i]

    mse = mean_squared_error(targets, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(targets, predictions)
    r2 = r2_score(targets, predictions)

    print(f"For {trait_names[i]}:")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R^2: {r2}\n")

<ipython-input-9-217b49c1381b>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(self.tweets[idx]['input_ids'], dtype=torch.long).squeeze(0)
<ipython-input-9-217b49c1381b>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(self.tweets[idx]['attention_mask'], dtype=torch.long).squeeze(0)


For Ext:
MSE: 0.09785258769989014
RMSE: 0.31281399726867676
MAE: 0.27574393153190613
R^2: -0.027681246933623394

For Neu:
MSE: 0.08247346431016922
RMSE: 0.28718194365501404
MAE: 0.236964151263237
R^2: -0.052290614779907596

For Agre:
MSE: 0.10090356320142746
RMSE: 0.31765320897102356
MAE: 0.2800156772136688
R^2: -0.024699146064368627

For Con:
MSE: 0.09510704129934311
RMSE: 0.30839428305625916
MAE: 0.2670899033546448
R^2: -0.03792363172586399

For Ope:
MSE: 0.10293589532375336
RMSE: 0.32083624601364136
MAE: 0.28078821301460266
R^2: -0.024732605968564858



In [14]:
df_predictions = pd.DataFrame(np.transpose(all_predictions), columns=trait_names)


correlation_matrix_predictions = df_predictions.corr()
print("\nCorrelation matrix for predicted values:")
print(correlation_matrix_predictions)


Correlation matrix for predicted values:
           Ext       Neu      Agre       Con       Ope
Ext   1.000000  0.163938  0.033777  0.192452  0.090574
Neu   0.163938  1.000000  0.366662  0.454421  0.252489
Agre  0.033777  0.366662  1.000000  0.216841  0.042806
Con   0.192452  0.454421  0.216841  1.000000  0.107238
Ope   0.090574  0.252489  0.042806  0.107238  1.000000


In [15]:
def color_cells(val):
    if val < 0:
        return 'background-color: red'
    elif val == 1:
        return 'background-color: yellow'
    else:
        return 'background-color: green'

styled_df = correlation_matrix_predictions.style.applymap(color_cells)

styled_df